In [1]:
import pandas as pd
import cv2
import numpy as np
import os
from tensorflow.keras.utils import to_categorical
import albumentations as A

def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x//2) - (min_dim//2)
  start_y = (y//2) - (min_dim//2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

augmented_video = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.GaussianBlur(p=0.2),
    A.ColorJitter(p=0.2)
])

def load_video(path, max_frames=16, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    
      frame = crop_center_square(frame)
      frame = cv2.resize(frame, resize)
      # frame = frame[:, :, [2, 1, 0]]
      augmented = augmented_video(image=frame)
      frame = augmented["image"]  
      frames.append(frame)

      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  # return (np.array(frames)/127.5) - 1.0
  return np.array(frames)/255.0 

# def augmented_video(video):
#   if np.random.random() > 0.5:
#       video = video[:, :, ::-1, :]
#   video = np.clip(video * np.random.uniform(0.8, 1.2), 0, 1)
#   video = np.clip(video * np.random.uniform(0.8, 1.2) + 0.5, 0, 1)
#   return video  

def augment_video(video):
  augmented_frames = []
  for frame in video:
      frame_uint8 = (frame*255).astype(np.uint8)
      augmented = augmented_video(image = frame_uint8)['image']
      augmented_frames.append(augmented.astype(np.float32)/255.0)
  return np.array(augmented_frames)  

def load_dataset(folder_path, max_frames=16, resize=(224, 224), augment_factor=2):
  X = []
  y = []
  class_names = sorted(os.listdir(folder_path))
  for idx, class_name in enumerate(class_names):
    class_folder = os.path.join(folder_path, class_name)
    for file_name in os.listdir(class_folder):
      video_path = os.path.join(class_folder, file_name)
      frames = load_video(video_path, max_frames=max_frames, resize=resize)
      if len(frames) == max_frames:
        X.append(frames)
        y.append(idx)
        for _ in range(augment_factor - 1):
            augmented = augment_video(frames.copy())
            X.append(augmented)
            y.append(idx)
  X = np.array(X)
  y = to_categorical(np.array(y), num_classes=len(class_names))
  return X, y, class_names

ModuleNotFoundError: No module named 'cv2'